In [1]:
import numpy as np

In [4]:
np.load("/app/_work/cache/audio_fragments/0/6/069c1ca39e372f9e293bdf317b7dbaeb.npy")

ValueError: cannot reshape array of size 518080 into shape (726176,)

In [5]:
! cp "/app/_work/cache/audio_fragments/0/6/069c1ca39e372f9e293bdf317b7dbaeb.npy" /app